# MNIST Handwritten Digit classification using Support Vector Machines

We will train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, we will need to use one-versus-all to classify all 10 digits. We also need to tune the hyperparameters using small validation sets to speed up the process.

Lets train  on a dataset of handwritten numbers, with labels to tell us what the numbers should be. MNIST has 60,000 training examples, and 10,000 for testing.

First, let's load the dataset and split it into a training set and a test set. We could use `train_test_split()` but we just take the first 60,000 instances for the training set, and the last 10,000 instances for the test set (this makes it possible to compare your model's performance with others): 

## Download MNSIT Data


In [107]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata("MNIST original")
X = mnist["data"]
y = mnist["target"]

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [108]:
y_train

array([ 0.,  0.,  0., ...,  9.,  9.,  9.])

Many training algorithms are sensitive to the order of the training instances, so it's generally good practice to shuffle them first:

In [109]:
np.random.seed(42)
rnd_idx = np.random.permutation(60000)
X_train = X_train[rnd_idx]
y_train = y_train[rnd_idx]

## Train a linear SVM classifier

Let's start simple, with a linear SVM classifier. It will automatically use the One-vs-All (also called One-vs-the-Rest, OvR) strategy, so there's nothing special we need to do. Easy!

In [110]:
#might take a minute or two to train
from sklearn.svm import LinearSVC
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

Let's make predictions on the training set and measure the accuracy (we don't want to measure it on the test set yet, since we have not selected and trained the final model yet):

In [111]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.85375000000000001

Wow, 82% accuracy on MNIST is a really bad performance. This linear model is certainly too simple for MNIST, but perhaps we just needed to scale the data first:

## Standardize the data and put in a pipeline

Standardize the data and put in a pipeline

In [112]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [113]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [67]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9224

In [122]:
#Data prep and baseline pipeline

minst_num_pipeline = Pipeline([
#        ('imputer', Imputer(strategy="median")), # - There are no missing values so imputer is not used
        ('std_scaler', StandardScaler()),
    ])


minst_base_pipeline = Pipeline([
        ("preparation", minst_num_pipeline),
        ("linear", LinearSVC(random_state = 42)),
        ])

In [123]:
minst_base_pipeline.fit(X_train.astype(np.float32), y_train)
y_pred_trn = minst_base_pipeline.predict(X_train.astype(np.float32))
print('Train Accuracy of LinearSVC: ', accuracy_score(y_train, y_pred_trn))

#minst_base_pipeline.fit(X_test)
#y_pred_tst = minst_base_pipeline.predict(X_test)
#print('Test Accuracy of LinearSVC: ', accuracy_score(y_test, y_pred_tst))

Train Accuracy of LinearSVC:  0.9204


## Use non-linear kernels (RBFs, Polynomial?)

Put everything in a **pipeline**.

That's much better (we cut the error rate in two), but still not great at all for MNIST. If we want to use an SVM, we will have to use a kernel. Let's try an `SVC` with an RBF kernel (the default).

Note: If we are using Scikit-Learn ≤ 0.19, the `SVC` class will use the One-vs-One (OvO) strategy by default, so we must explicitly set `decision_function_shape="ovr"` if we want to use the OvR strategy instead (OvR is the default since 0.19).

In [124]:
from sklearn.svm import SVC
svm_clf = SVC(decision_function_shape="ovr")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [125]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.94615000000000005

In [128]:
#Pipelie to fine tune SVC with RBF kernel
minst_SVC_pipeline = Pipeline([
        ("preparation", minst_num_pipeline),
        ("SVC", SVC(decision_function_shape="ovr")),
        ])

In [129]:
minst_SVC_pipeline.fit(X_train.astype(np.float32)[:10000], y_train[:10000])
y_pred_trn = minst_SVC_pipeline.predict(X_train.astype(np.float32))
print('Train Accuracy of SVC RBF: ', accuracy_score(y_train, y_pred_trn))

#minst_SVC_pipeline.fit(X_test)
#y_pred_tst = minst_SVC_pipeline.predict(X_test)
#print('Test Accuracy of SVC RBF: ', accuracy_score(y_test, y_pred_tst))

Train Accuracy of SVC RBF:  0.94485


## Tune hyperparameters

Put everything in a **pipeline**.

That's promising, we get better performance even though we trained the model on 6 times less data. Let's tune the hyperparameters by doing a randomized search with cross validation. We will do this on a small dataset just to speed up the process:

In [130]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(loc=0, scale=4)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])  #### use a subset of the data to speed things up

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=0.33087840419, gamma=0.00636473705545 .........................
[CV] .......... C=0.33087840419, gamma=0.00636473705545, total=   1.6s
[CV] C=0.33087840419, gamma=0.00636473705545 .........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] .......... C=0.33087840419, gamma=0.00636473705545, total=   1.5s
[CV] C=0.33087840419, gamma=0.00636473705545 .........................
[CV] .......... C=0.33087840419, gamma=0.00636473705545, total=   1.5s
[CV] C=3.55007967751, gamma=0.0513498334519 ..........................
[CV] ........... C=3.55007967751, gamma=0.0513498334519, total=   1.5s
[CV] C=3.55007967751, gamma=0.0513498334519 ..........................
[CV] ........... C=3.55007967751, gamma=0.0513498334519, total=   1.5s
[CV] C=3.55007967751, gamma=0.0513498334519 ..........................
[CV] ........... C=3.55007967751, gamma=0.0513498334519, total=   1.5s
[CV] C=2.23997163713, gamma=0.0599166657847 ..........................
[CV] ........... C=2.23997163713, gamma=0.0599166657847, total=   1.4s
[CV] C=2.23997163713, gamma=0.0599166657847 ..........................
[CV] ........... C=2.23997163713, gamma=0.0599166657847, total=   1.5s
[CV] C=2.23997163713, gamma=0.0599166657847 ..........................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.0min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000201E372E8D0>, 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000201FB30DBA8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [131]:
rnd_search_cv.best_estimator_

SVC(C=3.2213743901948475, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0035964905225331812,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [132]:
rnd_search_cv.best_score_

0.81200000000000006

This looks pretty low but we only trained the model on 1,000 instances. Let's retrain the best estimator on the whole training set (run this at night, it will take hours):

In [133]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=3.2213743901948475, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0035964905225331812,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [134]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.99968333333333337

This looks good! Let's select this model. Now we can test it on the test set:

In [135]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.96009999999999995

This reulst is not too bad, but apparently the model is overfitting slightly. It's tempting to tweak the hyperparameters a bit more (e.g. decreasing `C` and/or `gamma`), but we would run the risk of overfitting the test set. Other people have found that the hyperparameters `C=5` and `gamma=0.005` yield even better performance (over 98% accuracy). By running the randomized search for longer and on a larger part of the training set, we may be able to find this as well.

In [149]:
param_distributions1 = {"SVC__gamma": reciprocal(0.001, 0.1), "SVC__C": uniform(loc=0, scale=4)}

rnd_search_cv_SVCtune = RandomizedSearchCV(minst_SVC_pipeline, param_distributions1, n_iter=10, cv=3, verbose=2)
rnd_search_cv_SVCtune.fit(X_train.astype(np.float32)[:10000], y_train[:10000])

rnd_search_cv_SVCtune.best_estimator_.fit(X_train.astype(np.float32), y_train)

y_pred_SVCtune = rnd_search_cv_SVCtune.best_estimator_.predict(X_train.astype(np.float32))
acc = accuracy_score(y_train, y_pred_SVCtune)

print('Train Accuracy of SVC tuned: ', acc)



Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] SVC__C=2.34693833981, SVC__gamma=0.0541848283081 ................
[CV] . SVC__C=2.34693833981, SVC__gamma=0.0541848283081, total= 2.5min
[CV] SVC__C=2.34693833981, SVC__gamma=0.0541848283081 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[CV] . SVC__C=2.34693833981, SVC__gamma=0.0541848283081, total= 2.5min
[CV] SVC__C=2.34693833981, SVC__gamma=0.0541848283081 ................
[CV] . SVC__C=2.34693833981, SVC__gamma=0.0541848283081, total= 2.5min
[CV] SVC__C=1.29718889671, SVC__gamma=0.073882218826 .................
[CV] .. SVC__C=1.29718889671, SVC__gamma=0.073882218826, total= 2.5min
[CV] SVC__C=1.29718889671, SVC__gamma=0.073882218826 .................
[CV] .. SVC__C=1.29718889671, SVC__gamma=0.073882218826, total= 2.5min
[CV] SVC__C=1.29718889671, SVC__gamma=0.073882218826 .................
[CV] .. SVC__C=1.29718889671, SVC__gamma=0.073882218826, total= 2.5min
[CV] SVC__C=1.33080974845, SVC__gamma=0.0166112624505 ................
[CV] . SVC__C=1.33080974845, SVC__gamma=0.0166112624505, total= 2.3min
[CV] SVC__C=1.33080974845, SVC__gamma=0.0166112624505 ................
[CV] . SVC__C=1.33080974845, SVC__gamma=0.0166112624505, total= 2.3min
[CV] SVC__C=1.33080974845, SVC__gamma=0.0166112624505 ................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 92.9min finished


Train Accuracy of SVC tuned:  0.997033333333
